# SpatialIQ: Predicting Students' Spatial Intelligence through Machine Learning

## A Comprehensive AI-in-Education Research Study

**Author:** AI Research Developer  
**Date:** November 2025  
**Dataset Citation:** DOI: 10.21227/5qxw-bw66  
**Purpose:** Interpretable prediction and analysis of spatial intelligence in high school students using behavioral, academic, and demographic features

---

## Executive Summary

This notebook presents a rigorous machine learning pipeline designed to predict students' **Spatial Intelligence** levels (ranging from Very Low to Very High) using a rich dataset of 40 behavioral, academic, and demographic features collected from high school students. Spatial intelligence—the ability to visualize, manipulate, and reason about spatial relationships—is a critical component of cognitive development and academic achievement, particularly in STEM disciplines.

### Research Questions:
1. **What behavioral and academic patterns predict spatial intelligence?**
2. **How do demographic factors (gender, parental education, environment) influence spatial reasoning ability?**
3. **Can we build interpretable models that identify actionable insights for educators?**
4. **What ethical considerations arise from AI-driven student cognitive profiling?**

### Methodology Overview:
- **Phase 1:** Comprehensive data exploration and statistical profiling
- **Phase 2:** Intelligent feature engineering with domain knowledge
- **Phase 3:** Competitive modeling with Logistic Regression, Random Forest, XGBoost, and Neural Networks
- **Phase 4:** Deep model interpretation using SHAP explainability methods
- **Phase 5:** Actionable insights and ethical considerations

---

## Dataset Overview

The dataset comprises 398 high school students with 40 features across five categories:
- **Demographics:** Age, Gender, Class size, Environment (Urban/Suburban/Rural)
- **Socioeconomic:** Family size, Parental occupation, Parental education, Income level
- **Academic:** Major, GPA, Study time, Extra classes, Teacher assessment
- **Behavioral:** Internet usage, TV watching, Pattern recognition, Geographic familiarity
- **Gaming Preferences:** Action, Adventure, Strategy, Sport, Simulation, Role-playing, Puzzle games
- **Learning Modes:** Visual vs. Auditory, Map usage, Diagram usage, Experience with GIS

**Target Variable:** Spatial Intelligence (Categorical: VL, L, M, H, VH)


---

# Section 1: Import Required Libraries and Dataset Loading

This foundational section initializes all necessary libraries for data manipulation, visualization, machine learning, and model interpretation. We leverage industry-standard tools including pandas for data handling, scikit-learn for preprocessing and modeling, XGBoost for gradient boosting, TensorFlow for neural networks, and SHAP for model explainability. This comprehensive toolkit enables both rigorous statistical analysis and production-grade machine learning workflows.

In [ ]:
# ==================== LIBRARY IMPORTS ====================
# Core Data Science and Manipulation
import pandas as pd
import numpy as np
from pathlib import Path

# Data Preprocessing and Encoding
from sklearn.preprocessing import StandardScaler, LabelEncoder, OrdinalEncoder, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer

# Model Selection, Training, and Evaluation
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier, plot_importance
from lightgbm import LGBMClassifier

# Metrics and Evaluation
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score,
    roc_curve, auc, matthews_corrcoef
)

# Statistical Analysis
from scipy.stats import chi2_contingency, mutual_info_classif, spearmanr, pearsonr
from scipy.stats import kurtosis, skew
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Feature Importance and SHAP Interpretability
import shap
from sklearn.inspection import permutation_importance

# Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Warnings and Configuration
import warnings
warnings.filterwarnings('ignore')

# Configuration for visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✓ All libraries successfully imported")
print(f"✓ NumPy version: {np.__version__}")
print(f"✓ Pandas version: {pd.__version__}")
print(f"✓ Scikit-learn version: {pd.__version__.split('.')[0]}")
print(f"✓ SHAP library initialized for model interpretation")

## 1.1: Load and Inspect the Dataset

In this subsection, we load the Dataset.csv file and perform initial exploratory inspection to understand the data structure, dimensions, data types, and preliminary statistical characteristics. This foundational step is critical for identifying potential data quality issues, missing values, and the nature of features we'll be working with throughout the analysis.